In [11]:
import pandas as pd
import pyarrow
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [3]:
!curl https://raw.githubusercontent.com/oconnoj1/Lab_6_1/refs/heads/main/data/ReverseCodingItems.csv -o ./sample_data/ReverseCodingItems.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3162  100  3162    0     0   9598      0 --:--:-- --:--:-- --:--:--  9610


In [4]:
!curl https://raw.githubusercontent.com/oconnoj1/Lab_6_1/refs/heads/main/data/health_survey.csv -o ./sample_data/health_survey.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  219k  100  219k    0     0   517k      0 --:--:-- --:--:-- --:--:--  518k


In [5]:
(reverse :=
 spark.read.csv('./sample_data/ReverseCodingItems.csv', header=True)
)

DataFrame[Question: string, Construct: string, Question # on Qualtrics Survey: string, Needs Reverse Coding?: string, Column Name: string]

In [43]:
(survey :=
 spark.read.csv('./sample_data/health_survey.csv', header=True)
)
rename_dict = {c: c.replace(".", "_") for c in survey.columns}
survey = survey.rename([col(old).alias(new) for old, new in rename_dict.items()])

In [44]:
rename_dict

{'_c0': '_c0',
 'F1': 'F1',
 'F5': 'F5',
 'F2': 'F2',
 'F1.1': 'F1_1',
 'F2.1': 'F2_1',
 'F6': 'F6',
 'F4': 'F4',
 'F3': 'F3',
 'F5.1': 'F5_1',
 'F1.2': 'F1_2',
 'F2.2': 'F2_2',
 'F6.1': 'F6_1',
 'F2.3': 'F2_3',
 'F4.1': 'F4_1',
 'F2.4': 'F2_4',
 'F5.2': 'F5_2',
 'F2.5': 'F2_5',
 'F6.2': 'F6_2',
 'F1.3': 'F1_3',
 'F2.6': 'F2_6',
 'F5.3': 'F5_3',
 'F4.2': 'F4_2',
 'F2.7': 'F2_7',
 'F3.1': 'F3_1',
 'F2.8': 'F2_8',
 'F5.4': 'F5_4',
 'F3.2': 'F3_2',
 'F1.4': 'F1_4',
 'F3.3': 'F3_3',
 'F1.5': 'F1_5',
 'F5.5': 'F5_5',
 'F6.3': 'F6_3',
 'F1.6': 'F1_6',
 'F5.6': 'F5_6',
 'F2.9': 'F2_9',
 'F3.4': 'F3_4',
 'F4.3': 'F4_3',
 'F2.10': 'F2_10',
 'F1.7': 'F1_7',
 'F6.4': 'F6_4',
 'F4.4': 'F4_4',
 'F5.7': 'F5_7',
 'F3.5': 'F3_5',
 'F2.11': 'F2_11'}

In [41]:
survey

DataFrame[_c0: string, F1: string, F5: string, F2: string, F1.1: string, F2.1: string, F6: string, F4: string, F3: string, F5.1: string, F1.2: string, F2.2: string, F6.1: string, F2.3: string, F4.1: string, F2.4: string, F5.2: string, F2.5: string, F6.2: string, F1.3: string, F2.6: string, F5.3: string, F4.2: string, F2.7: string, F3.1: string, F2.8: string, F5.4: string, F3.2: string, F1.4: string, F3.3: string, F1.5: string, F5.5: string, F6.3: string, F1.6: string, F5.6: string, F2.9: string, F3.4: string, F4.3: string, F2.10: string, F1.7: string, F6.4: string, F4.4: string, F5.7: string, F3.5: string, F2.11: string]

In [37]:
(q_cols :=
 survey.drop(col('_c0')).columns
)

['F1',
 'F5',
 'F2',
 'F1.1',
 'F2.1',
 'F6',
 'F4',
 'F3',
 'F5.1',
 'F1.2',
 'F2.2',
 'F6.1',
 'F2.3',
 'F4.1',
 'F2.4',
 'F5.2',
 'F2.5',
 'F6.2',
 'F1.3',
 'F2.6',
 'F5.3',
 'F4.2',
 'F2.7',
 'F3.1',
 'F2.8',
 'F5.4',
 'F3.2',
 'F1.4',
 'F3.3',
 'F1.5',
 'F5.5',
 'F6.3',
 'F1.6',
 'F5.6',
 'F2.9',
 'F3.4',
 'F4.3',
 'F2.10',
 'F1.7',
 'F6.4',
 'F4.4',
 'F5.7',
 'F3.5',
 'F2.11']

In [38]:
(
    survey_stacked :=
    survey
    .unpivot(ids = '_c0',
             values = ['F1.1'],
             variableColumnName='Question',
             valueColumnName='Response')
).toPandas()

AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "F1". Need a complex type [STRUCT, ARRAY, MAP] but got "STRING".